# Advent of Code 2024 Day 11 

### Setup

In [2]:
from aocd import get_data, submit

day = 11
year = 2024


In [ ]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

In [ ]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

##### Data Parsing

In [ ]:
import numpy as np

def parse_data(raw_data:str):
    return np.array([ int(x) for x in raw_data.split() ])

sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

sample_data

### Part One!

In [6]:
use_sample_data = False
part = 'a'

In [ ]:
data = sample_data if use_sample_data else test_data

data

In [8]:
def blink(data: np.ndarray):
    result = []
    
    for x in data:
        xstring = str(x)

        if x == 0:
            result.append(1)

        elif x >= 10 and len(xstring) % 2 == 0:
            split = len(xstring) // 2
            left, right = int(xstring[:split]), int(xstring[split:])
            
            result += [left, right]
        
        else:
            result.append(x * 2024)
    
    return np.array(result)
        

In [ ]:
output = data.copy()
sizes = [output.size]
for _ in range(25):
    output = blink(output)
    sizes.append(output.size)


part_a_answer = output.size
part_a_answer

In [ ]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer, part='a', day=day, year=year, reopen=True)

### Part Two!

In [11]:
use_sample_data = False
part='b'

In [12]:
data = sample_data if use_sample_data else test_data

In [13]:
def blink_optimized(data: np.ndarray):
    data_str = data.astype(str)
    data_str_len = np.char.str_len(data_str)
    
    split_mask = np.logical_and(data >= 10, data_str_len % 2 == 0)
    split_indices = np.flatnonzero(split_mask)
    split_values = data_str[split_indices]
    split_offset = np.cumsum(split_mask)
    split_indices_norm = split_indices + split_offset[split_indices]

    output_size = data.size + np.sum(split_mask)
    output = np.zeros(output_size, dtype=int)

    for idx, val in zip(split_indices_norm, split_values):
        split = len(val) // 2
        left, right = int(val[:split]), int(val[split:])
        output[idx - 1] = left
        output[idx] = right

    zero_mask = data == 0
    zero_indices = np.flatnonzero(zero_mask)
    zero_indices_norm = zero_indices + split_offset[zero_indices]
    output[zero_indices_norm] = 1

    mul_mask = np.logical_not(np.logical_or(zero_mask, split_mask))
    mul_indices = np.flatnonzero(mul_mask)
    mul_indices_norm = mul_indices + split_offset[mul_indices]
    output[mul_indices_norm] = data[mul_indices] * 2024

    return output

In [ ]:
import heapq

def project_blink_size(num: np.ndarray, blinks:int, memo:dict = None):
    memo = memo if memo is not None else {}
    key = (blinks, num)
    count = 0
    
    if blinks == 0:
            return 1
        
    if key in memo:
        return memo[key]
    
    children = blink(np.array([num]))

    for child in children:
        count += project_blink_size(child, blinks - 1, memo)
    
    memo[key] = count
    return count

In [ ]:
count = 0
memo = {}
for num in data:
    count += project_blink_size(num, 75, memo=memo)

part_b_answer = count
part_b_answer 

In [ ]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)